<a href="https://colab.research.google.com/github/VindhyaSRajan/Master-s-Thesis/blob/master/k_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install jieba

In [0]:
import jieba
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
import nltk
# Define function for tokenize and lemmatizing
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords') 
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#First, let's remove the stopwords
stopwords = nltk.corpus.stopwords.words('english')
#stopwords = stopwords.extend(cat_list) 
more_stopwords = ['one','two','three','four','five','stars','star','lawn','mower','weber','blower','feeder','feeders','hose',
                  'reel','nozzle','pump','bird','sprinkler','liner','garden','pest','control','patio','black',
                  'decker','mobiles','book','trimmer','washer','power', 'tell','1st','uv','bistro','tub','fertiliser'
                  'pressure','battery','electric','chain','saw','weed','snow','squirrel','america',
                  'chair','home','kitchen','table','charger','blades','panels','birds','bug','mice',
                  'trap','fire','bugs','bug','mice','trimmer','washer','power','pressure','hummingbirds'
                  'electric','chain','saw','weed','snow','grill','brush','bbq','charcoal','smoker','flag',
                  'plant','outdoor','table','covers','buyer','birds','anyone','rain','crap','gauge','meets',
                  'wireless','replacement','part','solar','oem','panel','switch','tires','belt','met',
                  'us','issue','buck','knife','tree','soil','straps','ball','axe','lopper','hatchet','socks',
                  'lights','cake','tin','boots','towel','leaf','plants','seeds','bulbs','bulb','flower','gardening',
                  'gloves','oven','smokers','grilling','ant','fly','killer','bars','drainage','cedar','fiskars','pruners'
                  'sprouts','chipper','saucer','roller','cordless','Cages','gas','chime','wind','ants','flies','fiskars',
                  'pool','dont','squirrels','sofa','grass','bench','flowers','swing','cushions','hummingbird','holders',
                  'bamboo','clay','incubator','swing','flowers','bench','hammock','birdhouse','holders','planks','doesnt','didnt'
                  ]
stopwords.extend(more_stopwords)
# merged_df['summary'] = merged_df['summary'].astype(str) 
# merged_df['summary'] = merged_df.summary.str.replace("[_\-;%()|+&=*%.,!?:#$@\[\]/]", "").str.lower()
# merged_df.loc[:,'summary'] = merged_df['summary'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))
# merged_df['summary']

In [0]:
def load_articles():
    
    filename = '/content/drive/My Drive/Amazon_data/k-means.pkl'
    if os.path.exists(filename):
        with open(filename, 'rb') as in_data:
            articles = pd.read_csv(in_data)
            return articles

    # stop_words = [word.strip() for word in open('data/stop_words.txt', 'r', encoding='utf-8').readlines()]
    # print(stop_words)
    data = pd.read_csv('/content/drive/My Drive/Amazon_data/Amazon2014_merged_all.csv')
    
    print("Removing Stopwords")
    #data['summary'] = data['summary'].astype(str) 
    data['summary'] = data.summary.str.replace("[_\-;%()|+&=*%.,!?:#$@\[\]/]", "").str.lower()
    data.loc[:,'summary'] = data['summary'].astype(str).apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))
    # data['main_cat'] = data['main_cat'].apply(lambda x: " ".join([word for word in jieba.cut(x) if word not in stop_words]))
    # data['summary'] = data['summary'].apply(
    #     lambda x: " ".join([word for word in jieba.cut(x) if word not in stop_words]))
    # data['content'] = data['content'].apply(
    #     lambda x: " ".join([word for word in jieba.cut(str(x)) if word not in stop_words]))

    articles = []
    for summary in zip(data['summary'].tolist()):
                                       
        article = summary
        articles.append(article)

    with open(filename, 'wb') as out_data:
        pickle.dump(articles, out_data)

    return articles


articles=load_articles()

Removing Stopwords


In [0]:
#Removing empty strings from the list
articles = [i for i in articles if i != ('',)]
articles = [''.join(i) for i in articles]
# for i in articles:  #This was to check for the class of every list element
#    print(type(i))
articles

['good story',
 "stop can't stop last page",
 'good read',
 'routine mundane difficult stop reading',
 'keeps hooked',
 'great read',
 'great story',
 'scary',
 'fantastic read really enjoyed',
 'could much better',
 'satisfyingly unsettling',
 'good',
 'good read',
 'interesting',
 'enjoyed expect end',
 'great read',
 'give chance regret',
 'imagine sitting front porch suburban alabama',
 'good heart pounding read',
 'solid read lovers crime fiction',
 'happens various members families affects older man whole story think go',
 'absolute gem read',
 'great',
 'want put',
 "can't wait read next",
 'true face evil',
 'read 2',
 'may classic police stories future',
 'excellent debut',
 'put kept reading glad',
 'gripping new series',
 '"a craftily written murder mystery"',
 'nice product pole needs 6" longer stronger',
 'pretty sturdy gives great shade',
 'good product price',
 'changes',
 'great purchase',
 'get pay',
 'ordered 2 arrived damaged get',
 'looks good',
 'everything cheap s

max_df is used for removing terms that appear too frequently, also known as "corpus-specific stop words". For example:

max_df = 0.50, "ignore terms that appear in more than 50% of the documents"
Default max_df = 1.0, "ignore terms that appear in more than 100% of the documents". 

min_df is used for removing terms that appear too infrequently. For example:
min_df = 0.01 means "ignore terms that appear in less than 1% of the documents".
Default min_df = 1, "ignore terms that appear in less than 1 document". 

In [0]:
def transform(articles, n_features=10000):
    """
    TF-IDF Transformation
    :param articles:
    :param n_features:
    :return:
    """
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=n_features, min_df=0.01, use_idf=True)
    X = vectorizer.fit_transform(articles)
    return X, vectorizer

In [0]:
def train(X, vectorizer, true_k=15, mini_batch=False, show_label=False):
    """
    k-means
    :param X:
    :param vectorizer:
    :param true_k:
    :param mini_batch:
    :param show_label:
    :return:
    """
    if mini_batch:
        k_means = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                                  init_size=1000, batch_size=1000, verbose=False)
    else:
        k_means = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=1,
                         verbose=False)
    k_means.fit(X)
    if show_label:  
        print("Top terms per cluster:")
        order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
        terms = vectorizer.get_feature_names()
        # print(vectorizer.get_stop_words())
        for i in range(true_k):
            print("Cluster %d" % i, end='')
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind], end='')
            print()
    result = list(k_means.predict(X))
    print('Cluster distribution:')
    print(dict([(i, result.count(i)) for i in result]))
    return -k_means.score(X)

In [0]:
def plot_params():
    """
    :return:
    """
    articles = load_articles()
    print("%d docments" % len(articles))
    X, vectorizer = transform(articles, n_features=500)
    true_ks = []
    scores = []
    for i in range(3, 80, 1):
        score = train(X, vectorizer, true_k=i) / len(articles)
        print(i, score)
        true_ks.append(i)
        scores.append(score)
    plt.figure(figsize=(8, 4))
    plt.plot(true_ks, scores, label="error", color="red", linewidth=1)
    plt.xlabel("n_features")
    plt.ylabel("error")
    plt.legend()
    plt.show()

In [0]:
#plot_params()
def out():
    """
 
    :return:
    """

    #articles = load_articles()
    X, vectorizer = transform(articles, n_features=10000)
    score = train(X, vectorizer, true_k=6, show_label=True) / len(articles)
    print(score)


out()

Top terms per cluster:
Cluster 0 great works price quality worked buy little looks work cover
Cluster 1 nice works perfect like easy best well work quality use
Cluster 2 love product great little easy really use works perfect quality
Cluster 3 good product quality price buy works looks great fit cover
Cluster 4 product great quality nice price awesome works happy buy would
Cluster 5 excellent product quality price great cover buy easy works good
Cluster distribution:


In [0]:
plot_params()